In [1]:
import json
f = open("labels_v2.json")
label = json.load(f)
#ruled_based matcher
import spacy
#nlp = spacy.load("/Users/luis/Documents/GitHub/2022Summer/AI-Oriented-Recommendation-System/PreprocessingData/spaCy/rule_model_7Categories")

Init Plugin
Init Graph Optimizer
Init Kernel


In [2]:
nlp = spacy.load("en_core_web_sm")
import nltk 

In [91]:
from spacy import displacy
doc = nlp("comfort")
displacy.render(doc)

In [4]:
nltk.pos_tag(("graphic raverback tank").split(" "))

[('graphic', 'JJ'), ('raverback', 'NN'), ('tank', 'NN')]

In [3]:
pt = label.get("product_type")
st = label.get("sub_product_type")

In [4]:
for key in st[0].keys():
    print(key)

sandals
sneakers
heels
boots
shirts
sweaters/hoodies
jackets/coats
activewear
shorts
skirts
pants
jeans
dresses
swimwear
jewelries
bags
glasses
headwear
cosmetics
skin care
beauty tools
hair care
nail care


In [ ]:
#8 categories
shoes = set()
tops = set()
bottoms = set()
other_clothing = set()

beauty = set()
accessories = set()
homeware = set()
other = set()

In [5]:
#method to compare two string similarities
from difflib import SequenceMatcher
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [8]:
similar("hooded blanket", "blanket")

0.6666666666666666

In [9]:
from typing import List
def check_and_concat_punct(temp:List[str]) -> str:
    '''
    input: list, e.g. ['t', '-', 'shirt'], ['high', '-', 'rise', 'shoe']
    function: deal with the list with '-' punct
    output: "t-shirt" "high-rise shoe" "a-b-c color" 
    '''
    temp_string = ""
    if temp.__contains__('-') and len(temp) == 3:
        temp_string = ''.join(temp)
    elif temp.__contains__('-') and len(temp) > 3:
        
        pos = temp.index('-')
        #if there is wrong position of '-'
        if pos < 1:
            temp_string = ' '.join(temp)
            return temp_string
        
        puct_string = temp[pos - 1] + temp[pos] + temp[pos + 1]
        rest_string = ' '.join(temp[pos + 2:])
        temp_string = puct_string + ' '+ rest_string
    else:
        temp_string = ' '.join(temp)
    
    return temp_string 
    

In [12]:
'''
input: original string e.g. 'scandals', 'jackets/coats'ArithmeticError
funct: clean the string to the original words and put to list
output: ['scandal'], ['jacket', 'coats'] 
'''
def lemma_string(original_string:str) -> List:
    doc = nlp(original_string.lower())
    temp = []
    for token in doc:
        if token.text == '&' or token.text == 'and' or token.text == '/' or token.text == ',':
            continue
        if token.text == "glasses" or token.text == 'booty' or token.text == "sunglasses": #some of them should be represented in plural format
            temp.append(token.text)
        else:
            temp.append(token.lemma_)
    return temp


In [13]:
lemma_string("Jumpsuits & Rompers")

['jumpsuit', 'romper']

In [14]:
sub_cat_all_keys = [key for key in st[0].keys()]
sub_cat_shoes_keys = sub_cat_all_keys[0:4]
sub_cat_tops_keys= sub_cat_all_keys[4:7]
sub_cat_other_clothing_keys = sub_cat_all_keys[7:8] + sub_cat_all_keys[13:14]
sub_cat_bottoms_keys = sub_cat_all_keys[8:13]
sub_cat_acc_keys = sub_cat_all_keys[14: 19]
sub_cat_beauty_keys = sub_cat_all_keys[19: ]
print(sub_cat_shoes_keys)
print(sub_cat_tops_keys)
print(sub_cat_other_clothing_keys)
print(sub_cat_bottoms_keys)
print(sub_cat_acc_keys)
print(sub_cat_beauty_keys)



['sandals', 'sneakers', 'heels', 'boots']
['shirts', 'sweaters/hoodies', 'jackets/coats']
['activewear', 'swimwear']
['shorts', 'skirts', 'pants', 'jeans', 'dresses']
['jewelries', 'bags', 'glasses', 'headwear', 'cosmetics']
['skin care', 'beauty tools', 'hair care', 'nail care']


In [15]:
#This map the product name to main_7categories type
#will be like {"slide" : 1} 1 is mapping to shoes
#from nltk.stem import PorterStemmer
#lemmatizer = PorterStemmer() #since this is not accurate as spacy.lemmanizer, don't use it.
all_main = {key for key in pt[0].keys()}
print("First: \n",  all_main)
#set
all_sub_main = set()
#dict { 'shoes': {(all shoe types whatever it is the 2nd or 3rd class label)} }
products_to_all = {}
#dict { 'shoes': 1, ''}
main_categories_map_to_num = {}
#dict { 'heels': 1, 't-shirt': } specific item maps to the number
specific_products_map_to_num = {}
pt_index = 0
for key in pt[0].keys():
    product_name_2nd_list = pt[0].get(key)
    pt_key = key
    pt_value_set = set()
    
    for item_string in product_name_2nd_list:
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp)

        #update set
        all_sub_main.add(temp_string)
        #update the value in the pair[]
        pt_value_set.add(temp_string)
    
    #main_to_sub map
    products_to_all.update({pt_key:pt_value_set})
    #product_to_num map
    pt_index += 1
    main_categories_map_to_num.update({pt_key:pt_index})

main_categories_map_to_num.update({"other" : 8})
main_categories_map_to_num.update({"unknown" : 0})
#if key in [scandals, sneakers, heels, boots], or [boots, shirts, sweaters, jackets, coats]
#
print("Second: all main_categories\n", all_sub_main)

all_sub = set()
for key in st[0].keys():
    product_name_3rd_list = st[0].get(key) # get list of []
    for item_string in product_name_3rd_list: 
        #loop all item_string in order
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp)
        temp_list = lemma_string(key)
        temp_string2 = check_and_concat_punct(temp_list)
        all_sub.add(temp_string)
        all_sub.add(temp_string2)
print("Third: all sub catrgories\n", all_sub)


First: 
 {'shoes', 'accessories', 'other', 'homeware', 'beauty', 'other_clothing', 'tops', 'bottoms'}
Second: all main_categories
 {'wedge', 'collar', 'pillow', 'kimono', 'bandana', 'sock tight', 'clock', 'bottle', 'umbrella', 'pyrrha', 'bra', 'tank', 'tie pocket square', 'activewear', 'vest', 'intimate', 'shirt', 'glove', 'culotte', 'area rug', 'headphone', 'jean', 'phone case', 'heel', 'skirt', 'skateboard wheel', 'sandal', 'jewelry', 'canvas', 'boot', 'perfume', 'poncho', 'mask', 'wakesurfs', 'snowboard', 'blouse', 'food', 'nail care', 'skin care', 'blanket', 'textile', 'scarf', 'artwork', 'pant', 't-shirt', 'hair care', 'ceramic', 'knitwear', 'jogger', 'woodcarve', 'sleepwear', 'drinkware', 'book', 'candle', 'sneaker', 'home decoration', 'croptee', 'bralette', 'puzzle', 'cologne', 'camisole', 'bath robe', 'tank top', 'jacket', 'skateboard wax', 'card stationery', 'towel', 'skateboard tool', 'jumpsuit romper', 'scrunchie', 'raverback tank', 'snowshoe', 'short', 'flat shoe', 'watch',

In [16]:
'''
input: the list of all sub_categories, the key of the main_categories
function:put all sub_categories to the relative the main_categories set
output: none
'''
def update_products_to_all(string_list : List, main_key: str)->None:
    assert len(string_list) > 0
    assert products_to_all.get(main_key) is not None
    for item_string in string_list:
        
        temp = lemma_string(item_string)
        temp_string = check_and_concat_punct(temp) 
        products_to_all.get(main_key).add(temp_string)


In [17]:
main_categories_map_to_num

{'shoes': 1,
 'tops': 2,
 'bottoms': 3,
 'other_clothing': 4,
 'beauty': 5,
 'accessories': 6,
 'homeware': 7,
 'other': 8,
 'unknown': 0}

In [18]:
#add all sub categories to the main 8 categories.
for key in sub_cat_all_keys:#loop all keys in sub_main_categories, 
    if key in sub_cat_tops_keys:#if the key belongs to tops
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_tops_keys, 'tops')
    elif key in sub_cat_shoes_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_shoes_keys, 'shoes')
    elif key in sub_cat_bottoms_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_bottoms_keys, 'bottoms')
    elif key in sub_cat_other_clothing_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_other_clothing_keys, 'other_clothing')
    elif key in sub_cat_acc_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_acc_keys, 'accessories')
    elif key in sub_cat_beauty_keys:
        product_name_3rd_list = st[0].get(key)
        update_products_to_all(product_name_3rd_list + sub_cat_beauty_keys, 'beauty') 
    

In [19]:
products_to_all

{'shoes': {'ankle boot',
  'ankle strap heel',
  'boot',
  'canvas sneaker',
  'chelsea boot',
  'chunky sandal',
  'classic sneaker',
  'clog',
  'espadrille',
  'falt sandal',
  'flat shoe',
  'flip flop',
  'heel',
  'heel boot',
  'heel wedge',
  'high top sneaker',
  'ice gripper boot',
  'jelly',
  'kitten heel',
  'lace-up boot',
  'mule',
  'oxford loafer',
  'platform',
  'platform boot',
  'platform heel',
  'pump',
  'rain boot',
  'sandal',
  'skate shoe',
  'slide',
  'slip-on sneaker',
  'slipper',
  'sneaker',
  'snowshoe',
  'stilettos',
  'tall boot',
  'waterproof boot',
  'wedge',
  'wedge heel',
  'winter boot'},
 'tops': {'blazer',
  'blouse',
  'bomber jacket',
  'bra',
  'bralette',
  'button-down shirt',
  'button-up shirt',
  'camisole',
  'cardigan',
  'coat',
  'crop hoodie',
  'crop shirt',
  'cropped sweater',
  'croptee',
  'croptop',
  'denim jacket',
  'dress shirt',
  'drop tee',
  'graphic tee',
  'hoodie',
  'intimate',
  'jacket',
  'jacket coat',
  

In [20]:
'''
Build the hashmap all products has mapped to the number of categories
'''
for key in products_to_all.keys():
    map_number_of_this_key = main_categories_map_to_num.get(key)
    assert products_to_all.get(key) is not None
    #include the main_categories to number;
    specific_products_map_to_num.update({key: map_number_of_this_key})
    for item in products_to_all.get(key):
        specific_products_map_to_num.update({item: map_number_of_this_key})
len(specific_products_map_to_num)


329

In [21]:
all_cat = all_sub.union(all_sub_main).union(all_main)

In [22]:
#compare the products_to_all with all_cat
comp = set()
for val in products_to_all.values():
    comp = comp.union(val)

all_cat.difference(comp)

{'accessories',
 'beauty',
 'bottoms',
 'homeware',
 'other',
 'other_clothing',
 'shoes',
 'tops'}

In [23]:
len(all_cat)

329

In [24]:

for item in all_cat:
    if item not in specific_products_map_to_num.keys():
        print(item)


In [25]:
import pandas as pd
from string_grouper import match_strings, match_most_similar, \
    group_similar_strings, compute_pairwise_similarities


In [26]:
df = pd.read_csv("temp_not_finished.csv")
#df.insert(df.columns.get_loc('1st_class_label'), "product_type(modified)", '', allow_duplicates=True)
#df.insert(df.columns.get_loc('1st_class_label'), "match_most_similar_>80%_string", '', allow_duplicates=True)
#df.insert(df.columns.get_loc('1st_class_label'), "match_string_>60%_similarity", '', allow_duplicates=True)
#df.insert(df.columns.get_loc('1st_class_label'), "group_rep_product_type", '', allow_duplicates=True)
#df.rename(columns={"Unnamed: 0":"current_index", "index": "original_index"}, inplace=True)
df.columns

Index(['current_index', 'original_index', 'id', 'title', 'tags', 'body_html',
       'product_type', 'product_type(modified)',
       'match_most_similar_>80%_string', 'match_string_>60%_similarity',
       'group_rep_product_type', '1st_class_label', '2nd_class_label',
       '3rd_class_label', 'raw_text'],
      dtype='object')

In [177]:
#df.drop("Unnamed: 0", inplace=True, axis=1)

In [178]:

#df.to_csv("temp_not_finished.csv", index=False)

In [27]:
removable_words = ["mens", "womens", "girls", "boys", "graphic", "premium", "cozy", "comfort", "casual"]
def modify_product_type(original:str)-> str:
    ori_list = original.split(" ")
    res = []
    for e in ori_list:
        if e in removable_words:
            continue
        else:
            res.append(e)
    return " ".join(res)

In [28]:
for i in range(df.shape[0]):
    ori_word = df.loc[i, ['product_type']].values[0]
    df.loc[i, ['product_type(modified)']] = modify_product_type(ori_word)


In [29]:
df.squeeze()

,current_index,original_index,id,title,tags,body_html,product_type,product_type(modified),match_most_similar_>80%_string,match_string_>60%_similarity,group_rep_product_type,1st_class_label,2nd_class_label,3rd_class_label,raw_text
0,0,1,6750552359068,Hues Underwire One Piece,"{d-cup+,meta-size-chart-artesands-size-guide,m...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",one piece,one piece,NaN,NaN,NaN,0,0,0,Hues Underwire One Piece; d cup+ meta size cha...
1,1,3,7552752550106,Marble Malibu Set,"{crop,meta-size-chart-bound-size-guide,moderat...","<p data-mce-fragment=""1""><span>A Bond-Eye orig...",bikini top,bikini top,NaN,NaN,NaN,0,0,0,Marble Malibu Set; crop meta size chart bound ...
2,2,4,5918371152028,Sophia Bottom,"{color-patterned,high-waist,meta-size-chart-po...","<meta charset=""utf-8"">\n<p>Meet your newest gi...",bikini bottom,bikini bottom,NaN,NaN,NaN,0,0,0,Sophia Bottom; color patterned high waist meta...
3,3,6,5918535942300,Sonja Top,"{color-stripes,crop,meta-size-chart-power-2-th...","<meta charset=""utf-8"">\n<p>This retro revival ...",bikini top,bikini top,NaN,NaN,NaN,0,0,0,Sonja Top; color stripes crop meta size chart ...
4,4,7,6613956362396,Shirley Tie-Side,"{high-waist,meta-size-chart-power-2-the-flower...","<meta charset=""utf-8"">\n<p><meta charset=""utf-...",bikini bottom,bikini bottom,NaN,NaN,NaN,0,0,0,Shirley Tie Side; high waist meta size chart p...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17968,17968,22069,2482003738706,Dree Crop Hi Rise / Cameo,"{""Citizens of Humanity"",Denim}","<meta charset=""utf-8"">\n<p class=""product-deta...",pant,pant,NaN,NaN,NaN,0,0,0,Dree Crop Hi Rise / Cameo; Citizens of Humanit...
17969,17969,22070,2408859238482,Rako Hoops / Sterling Silver,"{Earrings,""Fay Andrada""}","<meta charset=""utf-8"">\n<p><span>Lightweight h...",jewelry,jewelry,NaN,NaN,NaN,0,0,0,Rako Hoops / Sterling Silver; Earrings Fay And...
17970,17970,22071,1487071740013,Jamie Hi Rise Classic Fit / Plume,"{AGOLDE,Bottoms,Denim}","<meta charset=""utf-8"">\n<p><span data-v-19a2cf...",pant,pant,NaN,NaN,NaN,0,0,0,Jamie Hi Rise Classic Fit / Plume; AGOLDE Bott...
17971,17971,22072,147355598868,Tait Earrings in Brass and Malachite,"{Earrings,Quarry}",<p><span>The Tait Earrings are detailed with a...,jewelry,jewelry,NaN,NaN,NaN,0,0,0,Tait Earrings in Brass and Malachite; Earrings...


In [30]:
pre_defiened_labels = pd.Series(list(all_cat), name="pre_defined_label")
most_similar_matches = match_most_similar( pre_defiened_labels, df["product_type(modified)"],\
    min_similarity = 0.80, ignore_index=False, replace_na=False)
most_similar_matches

,most_similar_index,most_similar_pre_defined_label
0,288.0,one piece
1,50.0,bikini top
2,86.0,bikini bottom
3,50.0,bikini top
4,86.0,bikini bottom
...,...,...
17968,222.0,pant
17969,200.0,jewelry
17970,222.0,pant
17971,200.0,jewelry


In [32]:
pre_defiened_labels

0                jelly
1         shoulder bag
2                wedge
3      slip-on sneaker
4          kitten heel
            ...       
324      chunky sandal
325         face brush
326         ankle boot
327          lip gloss
328           cleanser
Name: pre_defined_label, Length: 329, dtype: object

In [33]:
most_similar_matches = pd.concat([df["original_index"],df["product_type(modified)"], most_similar_matches], axis=1)
most_similar_matches.to_csv("(GS)most_similar_string.csv")
most_similar_matches.shape


(17973, 4)

In [34]:
matches = match_strings(pre_defiened_labels, df["product_type(modified)"],\
    min_similarity = 0.60, ignore_index = False, replace_na=False)
matches.to_csv("(GS)match_strings.csv")
matches.shape

(14837, 5)

In [35]:
grouped_data = group_similar_strings(
    df["product_type(modified)"],
    min_similarity=0.60,
    ignore_index = False,
    replace_na = False
)
grouped_data.to_csv("(GS)group_similar_strings.csv")
grouped_data.shape

(17973, 2)

### Build 4 categories label dictionary


In [36]:
#accessory, other, 
all_cat

{'accessories',
 'activewear',
 'air freshener',
 'ankle boot',
 'ankle strap heel',
 'anklet',
 'area rug',
 'artwork',
 'athletic boxer',
 'aviator glasses',
 'backpack',
 'bag',
 'baggy jean',
 'bandana',
 'base layer',
 'baseball hat',
 'bath bomb',
 'bath mat',
 'bath robe',
 'beauty',
 'beauty tool',
 'belt',
 'bikini bottom',
 'bikini top',
 'blanket',
 'blazer',
 'blouse',
 'blue light block glasses',
 'blush',
 'board short',
 'body wash shower gel',
 'bodycon dress',
 'bomber jacket',
 'book',
 'boot',
 'bootcut jean',
 'booty short',
 'bottle',
 'bottoms',
 'bra',
 'bracelet',
 'bralette',
 'bronzer',
 'brooch',
 'brush cleaner',
 'brush set',
 'button-down shirt',
 'button-up shirt',
 'camisole',
 'candle',
 'canteen',
 'canvas',
 'canvas sneaker',
 'cap',
 'capri',
 'card stationery',
 'cardigan',
 'cargo short',
 'cargos pant',
 'cat eye glasses',
 'ceramic',
 'chelsea boot',
 'chinos pant',
 'chunky sandal',
 'classic sneaker',
 'cleanser',
 'clip',
 'cloak',
 'clock',
 

In [37]:
all_cat.__contains__("jumpsuit romper")

True

In [86]:
similar("acc", "accessories")

0.42857142857142855

In [38]:
print(most_similar_matches.columns)
print(matches.columns)
print(grouped_data.columns)

Index(['original_index', 'product_type(modified)', 'most_similar_index',
       'most_similar_pre_defined_label'],
      dtype='object')
Index(['left_index', 'left_pre_defined_label', 'similarity',
       'right_product_type(modified)', 'right_index'],
      dtype='object')
Index(['group_rep_index', 'group_rep_product_type(modified)'], dtype='object')


In [39]:
most_similar_matches.columns.fillna()

Index(['original_index', 'product_type(modified)', 'most_similar_index',
       'most_similar_pre_defined_label'],
      dtype='object')

In [40]:
most_similar_matches.squeeze()

,original_index,product_type(modified),most_similar_index,most_similar_pre_defined_label
0,1,one piece,288.0,one piece
1,3,bikini top,50.0,bikini top
2,4,bikini bottom,86.0,bikini bottom
3,6,bikini top,50.0,bikini top
4,7,bikini bottom,86.0,bikini bottom
...,...,...,...,...
17968,22069,pant,222.0,pant
17969,22070,jewelry,200.0,jewelry
17970,22071,pant,222.0,pant
17971,22072,jewelry,200.0,jewelry


In [41]:
grouped_data.squeeze()

,group_rep_index,group_rep_product_type(modified)
0,0,one piece
1,6109,bikini bottoms
2,6109,bikini bottoms
3,6109,bikini bottoms
4,6109,bikini bottoms
...,...,...
17968,1311,pants
17969,5306,jewelry
17970,1311,pants
17971,5306,jewelry


In [42]:
import numpy as np
np.isnan(most_similar_matches.loc[17972, 'most_similar_index'].item())


True

In [43]:
i = most_similar_matches.at[66, "original_index"]

df.loc[df['original_index'] == i, ['match_most_similar_>80%_string']]


,match_most_similar_>80%_string
66,NaN


In [44]:
#fill up the 80%
empty_cells = 0
for row in range(most_similar_matches.shape[0]):
    index = most_similar_matches.loc[row, "original_index"]
    most_similar_pre_defined_label = most_similar_matches.loc[row, 'most_similar_pre_defined_label']
    most_similar_index = most_similar_matches.loc[row, ['most_similar_index']].item()
    
    if not np.isnan(most_similar_index):
        df.loc[df['original_index'] == index, ['match_most_similar_>80%_string']] = most_similar_pre_defined_label
    else:
        empty_cells += 1

In [45]:
empty_cells

8096

In [46]:
#fill up 60 %
index_similarity_map = {}
index_to_pre_map = {}
for row in range(matches.shape[0]):
    current_index = matches.loc[row, 'right_index']
    label = matches.loc[row, 'left_pre_defined_label']
    similarity = matches.loc[row, 'similarity']
    
    if index_similarity_map.get(current_index) == None:
        index_similarity_map.update({current_index:similarity})
        index_to_pre_map.update({current_index:label})
    elif index_similarity_map.get(current_index) >= similarity:
        index_similarity_map.update({current_index:similarity})
        index_to_pre_map.update({current_index:label})
    else: # similarity < then current, ignore
        continue


In [47]:
for key in index_to_pre_map.keys():
    df.loc[df['current_index'] == key, ['match_string_>60%_similarity']] = index_to_pre_map.get(key)

In [48]:
df.columns

Index(['current_index', 'original_index', 'id', 'title', 'tags', 'body_html',
       'product_type', 'product_type(modified)',
       'match_most_similar_>80%_string', 'match_string_>60%_similarity',
       'group_rep_product_type', '1st_class_label', '2nd_class_label',
       '3rd_class_label', 'raw_text'],
      dtype='object')

In [49]:
#fill up group representative of the label 
for row in range(grouped_data.shape[0]):
    df.loc[row, 'group_rep_product_type'] = grouped_data.loc[row, "group_rep_product_type(modified)"]

In [51]:
#Algorithm 2: if both 60% and 80% don't exist, just ignore, 
#             if the 80% string doesn't exist, compare the 60% string with 80% string similarity, to fill the 80%.
w = df.loc[66, ["match_most_similar_>80%_string"]].item()
print(w)
def is_nan_string(string):
    return string != string

is_nan_string(w)
for row in range(df.shape[0]):
    word_80 = df.loc[row, ["match_most_similar_>80%_string"]].item()
    word_60 = df.loc[row, ["match_string_>60%_similarity"]].item()
    
    if is_nan_string(word_80 ) and is_nan_string(word_60):
        continue
    elif is_nan_string(word_80) and not is_nan_string(word_60):
        pt_modified = df.loc[row, 'product_type(modified)']
        if similar(word_60, pt_modified) > 0.66:
            df.loc[row, "match_most_similar_>80%_string"] = word_60

nan


In [52]:
#map most_similar_label to number_of_categories using specific_product_to_num_map
for row in range(df.shape[0]):
    key = df.loc[row, ['match_most_similar_>80%_string']].item()
    if key != None and specific_products_map_to_num.get(key) != None:
        df.loc[row, ['1st_class_label']] = specific_products_map_to_num.get(key)
    else:
        df.loc[row, ['1st_class_label']] = 0
        

In [53]:
#(Finally Check √)if the 1st.class.label is unknown to do last check with the algorithm2
for row in range(df.shape[0]):
    key = df.loc[row, ["match_most_similar_>80%_string"]].item()
    if df.loc[row, ['1st_class_label']].item() == 0 and not is_nan_string(key):
        df.loc[row, ['1st_class_label']] = main_categories_map_to_num.get(key)

In [54]:
unknown = 0
total = df.shape[0]
shoes = 0
other_clothing = 0
tops = 0
bottoms = 0
beauty = 0
home = 0
acc = 0
other = 0
for row in range(df.shape[0]):
    n = df.loc[row, ['1st_class_label']].item()
    if n == 0:
        unknown += 1
    elif n == 1:
        shoes += 1
    elif n == 2:
        tops += 1
    elif n == 3:
        bottoms += 1
    elif n == 4:
        other_clothing += 1
    elif n == 5:
        beauty += 1
    elif n == 6:
        acc += 1
    elif n == 7:
        home += 1
    elif n == 8:
        other += 1

In [55]:
totalnum = total

In [56]:
print("unknown", unknown, "\tratio of all", unknown/total)
total = totalnum - unknown
print("\nshoes", shoes,"\tratio", shoes/total)
print("tops", tops, "\tratio", tops/total)
print("bottoms", bottoms, "\tratio", bottoms/total)
print("other_clothing", other_clothing,"\tratio", other_clothing/total)
print("beauty", beauty, "\tratio", beauty/total)
print("accessories", acc, "\tratio", acc/total)
print("homeware", home, "\tratio", home/total)
print("other", other, "\tratio", other/total)


unknown 5588 	ratio of all 0.3109108106604351

shoes 546 	ratio 0.04408558740411789
tops 4365 	ratio 0.35244247073072266
bottoms 1866 	ratio 0.15066612838110618
other_clothing 1390 	ratio 0.11223253936213161
beauty 560 	ratio 0.04521598708114655
accessories 3083 	ratio 0.24893015744852645
homeware 453 	ratio 0.036576503835284616
other 122 	ratio 0.00985062575696407


In [258]:
#June 15 2022
df.to_csv("(temp)first_class_label_done.csv", index=False)


In [233]:
print("blanket and blanket/towel: ", similar("blanket", "blanket/towel"))
print("accessory and accessories: ", similar("accessory", "accessories"))
print("top and tops", similar("tops", "top"))
print("tops and tops-casual", similar("tops", "tops-casual"))
print("denim and denim skirt", similar("denim", "denim skirt"))
print("dress and dresses", similar("dress", "dresses"))
print("accessories and accessories-scarves", similar('accessories', "accessories-scarves"))


blanket and blanket/towel:  0.7
accessory and accessories:  0.8
top and tops 0.8571428571428571
tops and tops-casual 0.5333333333333333
denim and denim skirt 0.625
dress and dresses 0.8333333333333334
accessories and accessories-scarves 0.7333333333333333
